# Visualize molecules which were included in benchmark

(Author: David L. Mobley)

The directory `set_v03_non_redundant` contains molecules used in the final benchmarking of OpenFF 1.2/OPLS3e as for our paper in this area, after removal of failures, and molecules which were in the OpenFF 1.2 training set. This generates a visualization of the final set of molecules.

## Grab list of molecules

In [2]:
from openeye import oechem


full_set_smiles = set()
ifile = 'set_v03_non_redundant/trim3_full_qcarchive.smi'
file = open(ifile, 'r')
text = file.readlines()
file.close()

#Make into canonical isomeric SMILES, unique
for line in text:
    smi = line.strip()
    #print(smi)
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smi)
    if mol.NumAtoms()>0:
        oechem.OEAssignAromaticFlags(mol)
        full_set_smiles.add(oechem.OEMolToSmiles(mol))

print("Obtained %s molecules" % len(full_set_smiles))

Obtained 3271 molecules


## Store set PDFS

In [3]:
# Generate PDF files visualizing the molecules

from openeye import oedepict
import shutil
import os
if os.path.isdir('molecule_PDFs'): shutil.rmtree('molecule_PDFs')
os.mkdir('molecule_PDFs')


oemols = []
for smi in full_set_smiles:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smi)
    oemols.append(mol)

itf = oechem.OEInterface()
PageByPage = True
suppress_h = True
rows = 10
cols = 6
ropts = oedepict.OEReportOptions(rows, cols)
ropts.SetHeaderHeight(25)
ropts.SetFooterHeight(25)
ropts.SetCellGap(2)
ropts.SetPageMargins(10)
report = oedepict.OEReport(ropts)
cellwidth, cellheight = report.GetCellWidth(), report.GetCellHeight()
opts = oedepict.OE2DMolDisplayOptions(cellwidth, cellheight, oedepict.OEScale_Default * 0.5)
opts.SetAromaticStyle(oedepict.OEAromaticStyle_Circle)
pen = oedepict.OEPen(oechem.OEBlack, oechem.OEBlack, oedepict.OEFill_On, 1.0)
opts.SetDefaultBondPen(pen)
oedepict.OESetup2DMolDisplayOptions(opts, itf)
for i, mol in enumerate(oemols):
    cell = report.NewCell()
    mol_copy = oechem.OEMol(mol)
    oedepict.OEPrepareDepiction(mol_copy, False, suppress_h)
    disp = oedepict.OE2DMolDisplay(mol_copy, opts)

    oedepict.OERenderMolecule(cell, disp)

oedepict.OEWriteReport("molecule_PDFs/molecules.pdf" , report)

True